#  Задача 1 

### 1.1 Импорт данных

In [1]:
import pandas as pd
import binascii
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr, spearmanr, mannwhitneyu, ttest_ind

In [2]:
df_start = pd.read_excel('Проект 4 исходник.xlsx')
df_start['time_view'] = pd.to_datetime(df_start['time_view'])
df_start.sort_values('time_view')

,id_view,time_view,id_group,id_city,nflag_order
3043,10457124180,2023-01-01 00:02:13.376,0,1,1
1105,10457160244,2023-01-01 00:02:25.620,1,3,1
9366,10457162957,2023-01-01 00:03:04.728,1,1,1
14669,10457178200,2023-01-01 00:05:37.484,0,1,0
3131,10457133790,2023-01-01 00:08:12.159,1,8,1
...,...,...,...,...,...
12054,10457174730,2023-01-31 23:50:46.254,0,1,1
2001,10457127969,2023-01-31 23:51:33.602,1,2,1
445,10457163593,2023-01-31 23:53:42.231,1,8,1
16724,10457140831,2023-01-31 23:56:52.597,0,1,0


In [3]:
df = df_start

In [4]:
city_dict = pd.read_excel('Проект 4 исходник.xlsx',sheet_name='city_dict')
city_dict.set_index('id_city',inplace=True)

In [5]:
df = df.merge(city_dict, on='id_city')
df.sort_values('time_view',inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df

,id_view,time_view,id_group,id_city,nflag_order,name_city
0,10457124180,2023-01-01 00:02:13.376,0,1,1,Москва
1,10457160244,2023-01-01 00:02:25.620,1,3,1,Новосибирск
2,10457162957,2023-01-01 00:03:04.728,1,1,1,Москва
3,10457178200,2023-01-01 00:05:37.484,0,1,0,Москва
4,10457133790,2023-01-01 00:08:12.159,1,8,1,Нижний Новгород
...,...,...,...,...,...,...
21934,10457174730,2023-01-31 23:50:46.254,0,1,1,Москва
21935,10457127969,2023-01-31 23:51:33.602,1,2,1,Санкт-Петербург
21936,10457163593,2023-01-31 23:53:42.231,1,8,1,Нижний Новгород
21937,10457140831,2023-01-31 23:56:52.597,0,1,0,Москва


In [7]:
def test_calc(r1, r2, alpha = 0.05):
    
    s,p = ttest_ind(r1,r2)
    
    if p < alpha:
        print("Гипотеза H0 не подтверждается: средние не равны")
    else:
        print("Гипотеза H0 подтверждается: средние равны")
    
    print("Среднее значение 1 ряда", r1.mean())
    print("Среднее значение 2 ряда", r2.mean())
    print("Разница средних = ", r1.mean()-r2.mean())
    print("P_value = ",p)
    return s, p

### 3.2 Общий анализ

In [8]:
test_calc(df[df['id_group']==0]['nflag_order'],df[df['id_group']==1]['nflag_order'])

Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.6150932939220395
Среднее значение 2 ряда 0.6757851165301899
Разница средних =  -0.060691822608150425
P_value =  5.1365957325220535e-21


(-9.41595134321716, 5.1365957325220535e-21)

Прирост очевидный, понижение цены увеличило конверсию просмотра в заказ.

### 3.3 Анализ по городам

In [9]:
success_city = []

for city in city_dict['name_city']:
    print('   ', city, ':')
    print('    Количество наблюдений общее:' ,              len(df.loc[df['name_city']==city]['nflag_order']))
    print('    Количество наблюдений контрольная группа:' , len(df[df['id_group']==0].loc[df['name_city']==city]['nflag_order']))
    print('    Количество наблюдений тестовая группа:'    , len(df[df['id_group']==1].loc[df['name_city']==city]['nflag_order']))
    s,p = test_calc(df[df['id_group']==0].loc[df['name_city']==city]['nflag_order'] , \
                    df[df['id_group']==1].loc[df['name_city']==city]['nflag_order'] )
    print()
    print()
    if p < 0.05:
        success_city.append(city)

print('Города, где AB тест показал себя успешно:', success_city)

    Москва :
    Количество наблюдений общее: 8849
    Количество наблюдений контрольная группа: 4405
    Количество наблюдений тестовая группа: 4444
Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.6172531214528945
Среднее значение 2 ряда 0.691944194419442
Разница средних =  -0.07469107296654753
P_value =  1.368947706664987e-13


    Санкт-Петербург :
    Количество наблюдений общее: 4959
    Количество наблюдений контрольная группа: 2431
    Количество наблюдений тестовая группа: 2528
Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.6083916083916084
Среднее значение 2 ряда 0.6791930379746836
Разница средних =  -0.07080142958307512
P_value =  1.8595141124883944e-07


    Новосибирск :
    Количество наблюдений общее: 3287
    Количество наблюдений контрольная группа: 1609
    Количество наблюдений тестовая группа: 1678
Гипотеза H0 подтверждается: средние равны
Среднее значение 1 ряда 0.6283405842137973
Среднее значение 2 ряда 0.6531585

### 3.3 Анализ по времени и городам

In [14]:
def is_peak_hour(hour):
    return 1 if hour in [7,8,9,10,17,18,19,20] else 0

In [15]:
df['peak_hour_flag'] = df['time_view'].dt.hour.apply(is_peak_hour)

In [16]:
print ('Час-пик:')
test_calc(df[df['id_group']==0].loc[df['peak_hour_flag']==1]['nflag_order'] , \
          df[df['id_group']==1].loc[df['peak_hour_flag']==1]['nflag_order'])
print ()
print ('Не час-пик:')
test_calc(df[df['id_group']==0].loc[df['peak_hour_flag']==0]['nflag_order'] , \
          df[df['id_group']==1].loc[df['peak_hour_flag']==0]['nflag_order'])

Час-пик:
Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.6051008968609866
Среднее значение 2 ряда 0.6770637268082818
Разница средних =  -0.07196282994729519
P_value =  1.4294522784350877e-10

Не час-пик:
Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.6200055111600992
Среднее значение 2 ряда 0.6751420070327292
Разница средних =  -0.05513649587263003
P_value =  2.723734415858154e-12


(-6.99724331712408, 2.723734415858154e-12)

In [17]:
success_city_ph = []
success_city_no_ph = []

for city in city_dict['name_city']:
    print('   ', city, ':')
    print ('Час-пик:')
    s,p = test_calc(df[df['id_group']==0].loc[df['peak_hour_flag']==1].loc[df['name_city']==city]['nflag_order'] , \
                    df[df['id_group']==1].loc[df['peak_hour_flag']==1].loc[df['name_city']==city]['nflag_order'] )
    print()
    if p < 0.05:
        success_city_ph.append(city)
    print ('Не час-пик:')
    s,p = test_calc(df[df['id_group']==0].loc[df['peak_hour_flag']==0].loc[df['name_city']==city]['nflag_order'] , \
                    df[df['id_group']==1].loc[df['peak_hour_flag']==0].loc[df['name_city']==city]['nflag_order'] )
    print()
    print()
    if p < 0.05:
        success_city_no_ph.append(city)


print('Города, где AB тест показал себя успешно в час-пик:', success_city_ph)
print('Города, где AB тест показал себя успешно в обычное время:', success_city_no_ph)

    Москва :
Час-пик:
Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.5955284552845529
Среднее значение 2 ряда 0.7137014314928425
Разница средних =  -0.11817297620828959
P_value =  1.3449585112087216e-11

Не час-пик:
Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.6282007511095937
Среднее значение 2 ряда 0.6812227074235808
Разница средних =  -0.053021956313987095
P_value =  1.8030823524055564e-05


    Санкт-Петербург :
Час-пик:
Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.5870393900889453
Среднее значение 2 ряда 0.6876484560570071
Разница средних =  -0.10060906596806174
P_value =  2.3018931356634527e-05

Не час-пик:
Гипотеза H0 не подтверждается: средние не равны
Среднее значение 1 ряда 0.6186131386861314
Среднее значение 2 ряда 0.67497034400949
Разница средних =  -0.05635720532335853
P_value =  0.0006634181415393023


    Новосибирск :
Час-пик:
Гипотеза H0 подтверждается: средние равны
Среднее значение 1

# 3.5 Итоги

In [18]:
print('Города, где AB тест показал себя успешно:', success_city)
print('Города, где AB тест показал себя успешно в час-пик:', success_city_ph)
print('Города, где AB тест показал себя успешно в обычное время:', success_city_no_ph)

Города, где AB тест показал себя успешно: ['Москва', 'Санкт-Петербург', 'Казань']
Города, где AB тест показал себя успешно в час-пик: ['Москва', 'Санкт-Петербург']
Города, где AB тест показал себя успешно в обычное время: ['Москва', 'Санкт-Петербург', 'Казань', 'Архангельск']


В общем и целом снижение цены увеличивает конверсию, НО статистическая разница есть только в трех городах, а если еще и дробить по час-пикам, то применять нововведение необходимо в Москве и Питере, и не для час-пиков в Казани и Архангельске.